# Obtain a List of S3 URLs for a GES DISC Collection Using the CMR API
### Authors: Chris Battisto, Alexis Hunzinger
### Date Authored: 1-31-22
### Date Updated: 2-3-23

### Timing

Exercise: 15 minutes

### Overview

This notebook demonstrates how to obtain a list of S3 URLs for desired cloud-hosted GES DISC granules using the [Commmon Metadata Repository (CMR) API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html).

### Prerequisites

This notebook was written using Python 3.8, and requires these libraries and files: 
- requests

Identify your data collection of interest and acquire its shortname or concept ID.
- These can be found on the collection's Dataset Landing Page on the GES DISC website. For example, this is the Dataset Landing Page for Hourly MERRA-2 SLV: https://disc.gsfc.nasa.gov/datasets/M2T1NXSLV_5.12.4/summary  

[ADD an edited image showing how to find shortname and concept ID]
Consider how to organize this in gesdisc-tutorials


## Import Libraries

In [45]:
import requests

## Create a Function for CMR Catalog Requests

In [46]:
def cmr_request(params):
    response = requests.get(url,
                        params=params,
                        headers={
                            'Accept': 'application/json',
                        }
                       )
    return response

## Search CMR Catalogs and Obtain S3 URLs

### Check that the CMR catalog can be accessed

If "200, CMR is accessible" is returned, the catalog can be accessed!

In [47]:
url = 'https://cmr.earthdata.nasa.gov/search/collections'

# Create our request for finding cloud-hosted granules, and check that we can access CMR
response = cmr_request({
                        'cloud_hosted': 'True',
                        'has_granules': 'True'
                        })

if response.status_code == 200:
    print(str(response.status_code) + ", CMR is accessible")
else:
    print(str(response.status_code) + ", CMR is not accessible, check for outages")

200, CMR is accessible


### Search CMR with your desired data collection's shortname or concept ID and desired date range

Using the collection's shortname or concept ID, we can obtain individual granules by querying https://cmr.earthdata.nasa.gov/search/granules. By querying a JSON response of the granules that we want, we can obtain each granule's S3 URL. 

Here, we will parse out an S3 URL for the AQUA AIRS IR + MW Level 2 CLIMCAPS dataset. 
- **Shortname**: SNDRAQIML2CCPRET
- **Concept ID**: C1693440798-GES_DISC

Our desried date range is 00:00:00 to 06:00:00 on January 1, 2016. 

In [48]:
url = 'https://cmr.earthdata.nasa.gov/search/granules'

short_name = 'SNDRAQIML2CCPRET'
concept_id = 'C1693440798-GES_DISC'

start_time = '2016-01-01T00:00:00Z'
end_time = '2016-01-01T06:00:00Z'

# OPTION 1: Using shortname
response = cmr_request({
                        'short_name': short_name,
                        'temporal': start_time+','+end_time,
                        'page_size': 200
                        })

# OPTION 2: Using concept ID
response = cmr_request({
                        'concept_id': concept_id,
                        'temporal': start_time+','+end_time,
                        'page_size': 200
                        })

granules = response.json()['feed']['entry']

### Identify the S3 URL from each granule response and save to a list

Now you can create an empty list and fill it with the S3 URL contained in each granule's response. 

In [49]:
s3_urls = []
for granule in granules:
    s3_urls.append(next((item['href'] for item in granule['links'] if item["href"].startswith("s3://")), None))

### View your list of S3 URLs

In [50]:
s3_urls

['s3://gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2015/365/SNDR.AQUA.AIRS_IM.20151231T2359.m06.g240.L2_CLIMCAPS_RET.std.v02_39.G.210425005357.nc',
 's3://gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2016/001/SNDR.AQUA.AIRS_IM.20160101T0005.m06.g001.L2_CLIMCAPS_RET.std.v02_39.G.210412145010.nc',
 's3://gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2016/001/SNDR.AQUA.AIRS_IM.20160101T0011.m06.g002.L2_CLIMCAPS_RET.std.v02_39.G.210412145010.nc',
 's3://gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2016/001/SNDR.AQUA.AIRS_IM.20160101T0017.m06.g003.L2_CLIMCAPS_RET.std.v02_39.G.210412145010.nc',
 's3://gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2016/001/SNDR.AQUA.AIRS_IM.20160101T0023.m06.g004.L2_CLIMCAPS_RET.std.v02_39.G.210412145010.nc',
 's3://gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2016/001/SNDR.AQUA.AIRS_IM.20160101T0029.m06.g005.L2_